In [1]:
import os 
import re
from utils import SpecialTokenizer, num_tokens_from_string, get_completion, align_prompt2_n_queries
import json
from itertools import islice
from tqdm import tqdm
import pickle 
import pandas as pd

In [55]:
data = json.load(open("../data/cc_books_clean_whole2_nonum.json", "r"))
#cc_book = data["cc_libro_I"]

In [ ]:
#heuristic prompt
prompt = "###Instruction:\n\nSei un esperto in materia di giurisprudenza. Formula {} domande possibili a partire dal seguente Testo. Le domande devono strettamente riguardare il contenuto del testo e null'altro. Restituisci esclusivamente le domande e null'altro. Numera ogni domanda formulata.\n\n###Testo:\n"
n_questions = 3

#model choice prompt
#prompt = "###Instruction:\n\nSei un esperto in materia di giurisprudenza. Formula quante più domande possibili a partire dal seguente Testo. Le domande devono strettamente riguardare il contenuto della risposta e null'altro. Restituisci esclusivamente le domande e null'altro. Numera ogni domanda formulata.\n\n###Testo:\n"

In [ ]:
# #trial on book 1
# tokenizer = SpecialTokenizer()
# query_passage = []
# data = cc_book
# for k in tqdm(data):
#     p = align_prompt2_n_queries(data[k], tokenizer, prompt)
#     answ = get_completion(p + data[k], "gpt-4o")
#     #loop over the generated queries and separate them
#     for query in answ.split("\n"):
#         query_passage.append((k, query, data[k]))
# #build a dataframe
# columns = ["Articolo di riferimento", "Query", "Passage"]
# df = pd.DataFrame(query_passage, columns = columns)

100%|██████████| 392/392 [17:10<00:00,  2.63s/it]


In [3]:
from itertools import  islice
data = json.load(open("../data/cc_books_clean_whole2_nonum.json", "r"))
data = dict(islice(data.items(), 1, None))
data.keys()

dict_keys(['cc_libro_II', 'cc_libro_III', 'cc_libro_IV', 'cc_libro_V', 'cc_libro_VI'])

In [4]:
#complete loop over all the books
tokenizer = SpecialTokenizer()
dfs = []
#process book
for n, book in enumerate(data, 2):
    print(f"Processing {book}...\n")
    query_passage = []
    #process articles
    for k in tqdm(data[book]):
        try:
            p = align_prompt2_n_queries(data[book][k], tokenizer, prompt)
            answ = get_completion(p + data[book][k], "gpt-4o")
            
            #loop over the generated queries and separate them
            for query in answ.split("\n"):
                query_passage.append((k, query, data[book][k].replace("\n", " ")))
        except:
            print(k)
            
    #build and save a dataframe for each book
    columns = ["Articolo di riferimento", "Query", "Passage"]
    df = pd.DataFrame(query_passage, columns = columns)
    df.to_csv(f"../results/cc_book{n}_qa.tsv", sep = "\t")
    print(f"Saved df for book{n}")
    dfs.append(df)
    assert len(df["Articolo di riferimento"].unique()) == len(data[book]), "Number of unique articles does not match number of inputs"




Processing cc_libro_II...



100%|██████████| 345/345 [11:25<00:00,  1.99s/it]


Saved df for book2
Processing cc_libro_III...



100%|██████████| 359/359 [13:18<00:00,  2.22s/it]


Saved df for book3
Processing cc_libro_IV...



...

Saved df for book4
Processing cc_libro_V...



100%|██████████| 623/623 [22:43<00:00,  2.19s/it]


Saved df for book5
Processing cc_libro_VI...



100%|██████████| 320/320 [09:59<00:00,  1.87s/it]

Saved df for book6


In [6]:
df1 = pd.read_csv("../qa_results/cc_book1_qa.tsv", sep = "\t")
dfs.insert(0, df1)
for n,i in enumerate(dfs,1):
    print(i.shape)

(1115, 3)
(874, 3)
(949, 3)
(2116, 3)
(2132, 3)
(890, 3)


In [7]:
with open("../results/all_qa_dfs.pkl", "wb") as outfile:
    pickle.dump(dfs, outfile)

In [ ]:
count = 0
for e,i in enumerate(dfs):
    for p in dfs[e]["Query"].tolist():
        if re.search(r" secondo il testo\?", p):
            count+=1
print(f"{count} queries contains ' secondo il testo?' to be removed")

315 queries contains ' secondo il testo?' to be removed


In [18]:
# df3 = pd.read_csv("cc_book1_qa.tsv", sep = "\t")

In [26]:
# df3["Passage"] = df3["Passage"].apply(lambda x: x.replace("\n", " "))
# df.to_csv("cc_book1_qa2.tsv", sep = "\t", index = False)


In [8]:
# df.to_csv("cc_book1_qa.tsv", sep = "\t", index = False)

In [29]:
# df = generate_queries(cc_book, prompt = prompt, subset = 10)

100%|██████████| 10/10 [00:48<00:00,  4.83s/it]
